<a href="https://colab.research.google.com/github/Byeon-MJ/DL_Computer_Vision/blob/main/Video_Parking_Car_Count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import cv2
import numpy as np
import time
import io
import base64
from IPython.display import HTML

In [ ]:
# 원본 동영상 Display
video = io.open('/content/gdrive/MyDrive/CV/Parking Car Count/video/parking.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="30%" controls>
    <source src="data:video/mp4;base64, {0}" type="video/mp4"/>
    </video>'''.format(encoded.decode('ascii')))

In [ ]:
# Detection 할 원본 동영상
file_name = '/content/gdrive/MyDrive/CV/Parking Car Count/video/parking.mp4'
min_confidence = 0.5
output_name = 'parking_car_count_video.mp4' # Detection 된 output 동영상
elapsed_time = 0                            # 총 경과시간 초기화

In [ ]:
# Load Yolo
net = cv2.dnn.readNet('/content/gdrive/MyDrive/CV/Object Detection_YOLO/yolov3.weights',
                      '/content/gdrive/MyDrive/CV/Object Detection_YOLO/yolov3.cfg')

classes = []
with open('/content/gdrive/MyDrive/CV/Object Detection_YOLO/coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
def detectAndDisplay(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.9, fy=0.9)
    height, width, channels = img.shape

    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    confidences = []    # detection 한 Class의 신뢰도 지정하는 배열 정의
    boxes = []          # detection 한 boxing 정보를 지정하는 배열 정의

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)    # detection 한 Class id
            confidence = scores[class_id]   # detection 한 Class의 신뢰도

            # Filter only 'car'
            if class_id == 2 and confidence > min_confidence:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])              # boxing 정보를 boxes 배열에 저장
                confidences.append(float(confidence))   # 신뢰도를 confidences 배열에 저장

    # apply non-max suppression
    # 박스안에 노이즈를 하나로 만들기
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)

    print(f'Min Confidence : {min_confidence}')
    for i in range(len(boxes)):
        if i in indexes:    # 노이즈가 제거된 박스만 표시
            x, y, w, h = boxes[i]
            label = f'{confidences[i]:,.2%}'    # 신뢰도(확률)
            print(i, label)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 1)
            cv2.rectangle(img, (x, y - 20), (x + w, y), (0, 255, 0), -1)
            cv2.putText(img, label, (x + 10, y - 5), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 1)

    text = f'number of parked cars : {len(indexes)}'
    cv2.putText(img, text, (40, 40), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 10)
    cv2.putText(img, text, (40, 40), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)

    process_time = time.time() - start_time
    global elapsed_time
    elapsed_time += process_time
    print(f'= = = process time {process_time:.3f} seconds')

    # video 쓰기
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        writer = cv2.VideoWriter(output_name, fourcc, 30,
                                 (img.shape[1], img.shape[0]), True)
        
    # Disk에 Frame write
    if writer is not None:
        writer.write(img)

In [ ]:
# 원본 동영상에서 video stream 읽기
cap = cv2.VideoCapture(file_name)
writer = None

if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print(f'elapsed time {elapsed_time:.3f} seconds')
        break
    detectAndDisplay(frame)